In [4]:

def languageClassifier(filename, doc2vec):
    
    #loading data
    data = pd.read_csv(filename)
    d2v = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    #print(d2v)
    
    
    #combining doc2vec and language into a df
    languages = data["Language"]
    #print(d2v.shape)
   # print(languages)
    d2v["Language"] = languages
    #print(d2v)
    count = 0

    
    #sorting values by language, pruning NaNs
    knownLanguage = d2v.dropna()
    languages = languages.dropna()
    
    
    #sorting remaining instances by language
    data_by_lang = [(x,y.iloc[:,:]) for x, y in knownLanguage.groupby(languages)]
    
    #calculating centroids of each language
    langCentroids = []
    for language, df in data_by_lang:
        df.pop("Language")
        df.loc[language] = df.mean()
        df = df.iloc[-1]
        
        langCentroids.append(df)
        
    
    i = 0
    
   
    while i < len(d2v.index):
        
        if pd.isna(d2v.at[i, "Language"]):
            
            currRow = d2v.iloc[i].iloc[:-1]
            
            bestLang = langClass(langCentroids, currRow)
            d2v.at[i, "Language"] = bestLang
        i += 1
    
    
    
    bestLang = langClass(langCentroids, currRow)
    #print(currRow)
   # print(langCentroids)
    print(bestLang)
    print(d2v)
   
    return d2v

doc2vecLangs = languageClassifier("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")

NameError: name 'pd' is not defined

In [5]:
#################distance and similarity measures used ################

def cosine_sim(v1, v2):
    #note v1 and v2 are pandas series of equal length
    
    i = 0
    dotProduct = 0
    absV1 = 0
    absV2 = 0
    
    while i < len(v1):
        dotProduct += v1[i]*v2[i]
        absV1 += v1[i]**2
        absV2 += v2[i]**2
        
        i+=1
        
    absV1 = np.sqrt(absV1)
    absV2 = np.sqrt(absV2)
    return dotProduct / (absV1 * absV2)

def manDistance(v1, v2):
    
    i = 0
    totalDistance = 0
    while i < len(v1):
        totalDistance = np.abs(v1[i] - v2[i])
        i += 1
        
    return totalDistance

In [6]:
# functions for finding the best language class, first is more robust

def langClass(centroids, row, mode = "cosine"):
    
    simScores = {}
    print(row.name)
    for centroid in centroids:
        if mode == "cosine":
            simScores[centroid.name] = cosine_sim(centroid, row)
        else:
            simScores[centroid.name] = manDistance(centroid, row)
        
    #setting eng as base lang
    maxSim = simScores["fre"]
    bestLang = "fre"
    for lang, sim in simScores.items():
        
        #print(f"lang is {lang}, sim is {sim}")
        
        if mode == "cosine":
            if sim > maxSim:
                maxSim = sim
                bestLang = lang
                
        else:
            if sim < maxSim:
                maxSim = sim
                bestLang = lang
    return bestLang


def langClass2(centroids, row):
    
    simScores = {}
    for centroid in centroids:
         simScores[centroid.name] = manDistance(centroid, row)
        
    #setting eng as base lang
    maxSim = simScores["fre"]
    bestLang = "fre"
    for lang, sim in simScores.items():
        
        #print(f"lang is {lang}, sim is {sim}")
    
        if sim < maxSim:
            maxSim = sim
            bestLang = lang
            
    return bestLang


In [7]:
print("Language Counts with Cosine Similarity")
print(manDistClass["Language"].value_counts())

print("\nLanguage Counts with Manhattan Distance")

print(manDist2Class["Language"].value_counts())


Language Counts with Cosine Similarity


NameError: name 'manDistClass' is not defined

In [8]:
'''Same as previous one, but using Manhattan distance, very slow,least reliable'''

def languageClassifier2(filename, doc2vec):
    
    #loading data
    data = pd.read_csv(filename)
    d2v = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    #print(d2v)
    
    
    #combining doc2vec and language into a df
    languages = data["Language"]
    #print(d2v.shape)
   # print(languages)
    d2v["Language"] = languages
    #print(d2v)
    count = 0

    
    #sorting values by language, pruning NaNs
    knownLanguage = d2v.dropna()
    languages = languages.dropna()
    
    
    #sorting remaining instances by language
    data_by_lang = [(x,y.iloc[:,:]) for x, y in knownLanguage.groupby(languages)]
    
    #calculating centroids of each language
    langCentroids = []
    for language, df in data_by_lang:
        df.pop("Language")
        df.loc[language] = df.mean()
        df = df.iloc[-1]
        
        langCentroids.append(df)
        
    
    i = 0
    
   
    while i < len(d2v.index):
        
        if pd.isna(d2v.at[i, "Language"]):
            
            currRow = d2v.iloc[i].iloc[:-1]
            
            bestLang = langClass2(langCentroids, currRow)
            d2v.at[i, "Language"] = bestLang
        i += 1
    
    
    
    
    #print(currRow)
   # print(langCentroids)
    print(bestLang)
    print(d2v)
   
    return d2v


manDist2Class = languageClassifier2("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")

NameError: name 'pd' is not defined

In [ ]:
'''Uses Cosine similarity as metric, bad, takes too long, seems not good'''
def languageClassifier(filename, doc2vec):
    
    #loading data
    data = pd.read_csv(filename)
    d2v = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    #print(d2v)
    
    
    #combining doc2vec and language into a df
    languages = data["Language"]
    #print(d2v.shape)
   # print(languages)
    d2v["Language"] = languages
    #print(d2v)
    count = 0

    
    #sorting values by language, pruning NaNs
    knownLanguage = d2v.dropna()
    languages = languages.dropna()
    
    
    #sorting remaining instances by language
    data_by_lang = [(x,y.iloc[:,:]) for x, y in knownLanguage.groupby(languages)]
    
    #calculating centroids of each language
    langCentroids = []
    for language, df in data_by_lang:
        df.pop("Language")
        df.loc[language] = df.mean()
        df = df.iloc[-1]
        
        langCentroids.append(df)
        
    
    i = 0
    
   
    while i < len(d2v.index):
        
        if pd.isna(d2v.at[i, "Language"]):
            
            currRow = d2v.iloc[i].iloc[:-1]
            
            bestLang = langClass(langCentroids, currRow)
            d2v.at[i, "Language"] = bestLang
        i += 1
    
    
    
    bestLang = langClass(langCentroids, currRow)
    #print(currRow)
   # print(langCentroids)
    print(bestLang)
    print(d2v)
   
    return d2v

doc2vecLangs = languageClassifier("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")

In [20]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


'''Seems to be a good model, has good counts, verified a few instances seems to perform great in fact'''
def learnLanguage(file, doc2vec):
    
    
    data = pd.read_csv(file)
    X = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    y = data["Language"]
    
    y = pd.Series(y)
    # Assuming you have a feature matrix `X` and a target variable `y`
    # X should contain other features like doc2vec and word frequency counts
    # y should contain the language labels (with missing values)

    # Split the dataset into instances with and without missing language values
    X_with_language = X[~y.isnull()]
    y_with_language = y[~y.isnull()]
    X_missing_language = X[y.isnull()]

    # Split the dataset with language into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_with_language, y_with_language, test_size=0.2, random_state=42)

    # Train a Random Forest classifier on the instances with language
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    # Predict the missing language values using the trained model
    imputed_language = rf.predict(X_missing_language)

    # Merge the imputed language values with the original dataset
    y[y.isnull()] = imputed_language

    # Now you can proceed with your machine learning algorithm using the complete dataset
    return y

lol = learnLanguage("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")


In [29]:
print("Language counts with Random-Forest classifier")
print(lol.value_counts())


Language counts with Random-Forest classifier
eng    22555
spa      217
fre      175
ger       65
jpn       10
per        8
mul        7
por        5
lat        4
ita        4
zho        3
grc        2
heb        2
rus        2
ara        1
swe        1
frs        1
nld        1
Name: Language, dtype: int64


In [9]:
#with pd.option_context('display.max_rows', None,
#                       'display.max_columns', None,
#                       'display.precision', 3,
#                       ):
#    display(lol)